In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

import csv 
import matplotlib.pyplot as plt

import pickle
import random
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
import h5py
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
df = pd.read_csv("../input/bankmarketing/bank marketing.csv")
list(df.columns)

In [ ]:
df.drop(['job', 'marital','education','contact','month','poutcome'], axis = 1, inplace=True) 

In [ ]:
list(df.columns)

In [ ]:
data = df.values
X = data[:,0:10]
Y = data[:,10]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
model = Sequential()
model.add(Dense(20, activation='sigmoid', input_shape=(10,)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
!pip install cma

In [ ]:
#Implementing CMA-ES 
import cma
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
def cal_fitness(x):
    weight_layer_1 = np.array(x[0:200]).reshape(10,20)
    biases_layer_1 = np.array(x[200:220])
    weight_layer_2 = np.array(x[220:240]).reshape(20,1)
    biases_layer_2 = np.array(x[240:241])

    model.layers[0].set_weights([weight_layer_1,biases_layer_1])
    model.layers[1].set_weights([weight_layer_2,biases_layer_2])
    predictions = model.predict(X_train)

    return average_precision_score(y_train, predictions)

gen = 0
best_fitness = []
average_fitness = []
best_solution = {}
while(gen < 100):
    accuracyScores = []
    #while(generation < 1):
    bestever = cma.optimization_tools.BestSolution()
    es = cma.CMAEvolutionStrategy(241*[0],0.5, {'popsize':100, 'verb_append': bestever.evalsall})
    (solutions, func_values) = es.ask_and_eval(fitfunc)
    average_fitness.append(sum(func_values)/ len(func_values))
    best_fitness.append(max(func_values))
    index = func_values.index(best_fitness[gen])
    if(not best_solution):
        best_solution['key'] = best_fitness[gen]
        best_solution['value'] = solutions[index]
        best_solution['generation'] = generation
        best_solution['average'] = average_fitness[gen]
    if(best_solution and best_solution['key'] < best_fitness[gen]):
        best_solution['key'] = best_fitness[gen]
        best_solution['value'] = solutions[index]
        best_solution['generation'] = gen
        best_solution['average'] = average_fitness[gen]
   # print(func_values)
    print(best_fitness, index)
   # print(average_fitness)
    gen += 1
    
print(best_solution)

In [ ]:
#Feed the best solution 
weight_layer_1 = np.array(best_solution['value'][0:200]).reshape(10,20)
biases_layer_1 = np.array(best_solution['value'][200:220])
weight_layer_2 = np.array(best_solution['value'][220:240]).reshape(20,1)
biases_layer_2 = np.array(best_solution['value'][240:241])
model.layers[0].set_weights([weight_layer_1,biases_layer_1])
model.layers[1].set_weights([weight_layer_2,biases_layer_2])
predictions = model.predict(X_test, verbose=0)
prediction_classes = model.predict_classes(X_test,verbose = 0 )


print("this is the Mean Squared Error: {}".format(average_precision_score(y_test,predictions)))

print("this is the confusion matrix")
print(confusion_matrix(y_test,prediction_classes))
accuracyScores.append(accuracy_score(y_test, prediction_classes))
print('Accuracy Score', accuracyScores[0])
print(accuracyScores)